In [45]:
from pymongo import MongoClient
from pymongo import errors
from pprint import pprint
import numpy as np
import re

In [46]:
def connect_database():
    client = MongoClient('localhost', 27017)
    return client

In [47]:
client = connect_database()

In [48]:
client.list_database_names()

['Kitchen_test',
 'Perfume_Database',
 'WorldDevelopmentIndicators',
 'admin',
 'config',
 'local']

In [49]:
perfume_database = client.Perfume_Database

In [50]:
pprint(perfume_database.list_collection_names())

['Cleared_Crawled_Perfumes',
 'Scraped_Perfumes',
 'Extra01',
 'Perfumes',
 'Extra02',
 'Fragrances',
 'Crawled_Perfumes']


In [51]:
extra02_collection = perfume_database.Extra02
fragrances_collection = perfume_database.Fragrances

# Compare the data sets

In [52]:
res = extra02_collection.aggregate([
  { "$sample": { "size": 5 } }
])

for i in res:
    pprint(i)

{'_id': ObjectId('65c3ecc62be6a0a6d64f677a'),
 'base_notes1': 'Pear',
 'base_notes2': 'Melon',
 'base_notes3': 'Musk',
 'base_notes4': 'Rose',
 'base_notes5': 'Peony',
 'brand': 'Maison SONO',
 'concentration': 'EDP',
 'followers': 2115,
 'instagram_link': 'https://www.instagram.com/maisonsono/?hl=en',
 'mid_notes1': 'Red Apple',
 'mid_notes2': 'Patchouli',
 'mid_notes3': 'Vanilla',
 'mid_notes4': 'Sandalwood',
 'price': '169,000.00',
 'price_per_ml': '4,828.57',
 'size': 35,
 'top_notes1': 'Water Jasmine',
 'top_notes2': 'Ginger Flower',
 'top_notes3': 'Green Mandarin',
 'variant': 'Walking The Aisle',
 'variant_link': 'https://shopee.co.id/maisonsono/17668723027',
 'year': 2022}
{'_id': ObjectId('65c3ecc62be6a0a6d64f678c'),
 'base_notes1': 'Musk',
 'base_notes2': 'Amber',
 'base_notes3': 'Peach',
 'base_notes4': 'Caramel',
 'base_notes5': 'Vanilla',
 'brand': 'Onja',
 'concentration': 'EDP',
 'followers': 715,
 'instagram_link': 'https://www.instagram.com/onja.id/',
 'mid_notes1': 'F

In [53]:
pprint(fragrances_collection.find_one())

{'_id': ObjectId('65a54ac72b32912eefb9ea54'),
 'base notes': ['Civet', 'Resins', 'Musk'],
 'company': 'Bogue',
 'description': 'Maai by Bogue is a Floral Woody Musk fragrance for women and '
                'men. Maai was launched in 2014. The nose behind this '
                'fragrance is Antonio Gardoni. Top notes are Aldehydes and '
                'Tuberose; middle notes are Jasmine and Rose; base notes are '
                'Civet, Resins and Musk. .',
 'gender': 'for women and men',
 'gender_vote': {'average': 0.2, 'votes_count': 130},
 'image': 'https://fimgs.net/mdimg/perfume/375x500.26681.jpg',
 'longevity': {'eternal': 69,
               'long lasting': 140,
               'moderate': 28,
               'very weak': 23,
               'weak': 9},
 'main accords': {'aldehydic': 71.419,
                  'amber': 70.8567,
                  'animalic': 80.9679,
                  'balsamic': 44.2436,
                  'fresh': 57.3066,
                  'musky': 100.0,
        

# gather notes in their own array

In [54]:
document_into_array = {
        '$project': {
            'fields': {
                '$objectToArray': '$$ROOT'
            }
        }
    }

In [55]:
def filter_fields(regex_filter):
    return {
        '$project': {
            'fieldsArray': {
                '$filter': {
                    'input': '$fields', 
                    'as': 'item', 
                    'cond': {
                        '$regexMatch': {
                            'input': '$$item.k', 
                            'regex': regex_filter
                        }
                    }
                }
            }
        }
    }

In [56]:
def accumulate_field_values(field_name):
    return {
        '$project': {
            field_name: {
                '$map': {
                    'input': '$fieldsArray', 
                    'as': 'item', 
                    'in': '$$item.v'
                }
            }
        }
    }

Insert the new field to the dataset

In [57]:
def merge_calculated_field(): 
    return {
        '$merge': {
            'into': 'Extra02', 
            'on': '_id', 
            'whenMatched': 'merge', 
            'whenNotMatched': 'discard'
        }}

In [58]:
accumulate_top_notes_pipeline = [
    document_into_array,
    filter_fields("^top"),
    accumulate_field_values("top notes"),
    merge_calculated_field()
]

In [59]:
accumulate_middle_notes_pipeline = [
    document_into_array,
    filter_fields("^mid"),
    accumulate_field_values("middle notes"),
    merge_calculated_field()
]

In [60]:
accumulate_base_notes_pipeline = [
    document_into_array,
    filter_fields("^base"),
    accumulate_field_values("base notes"),
    merge_calculated_field()
]

## Accumulate new fields and merge

accumulate top fields and using the aggregation pipeline merge it with Extra02

In [63]:
result = extra02_collection.aggregate(accumulate_top_notes_pipeline)

In [65]:
extra02_collection.find_one()

{'_id': ObjectId('65c3ecc62be6a0a6d64f6747'),
 'brand': 'Alchemist',
 'instagram_link': 'https://www.instagram.com/alchemist.fragrance/',
 'followers': 17800,
 'year': 2022,
 'variant': 'Powder Room',
 'variant_link': 'https://shopee.co.id/alchemist.fragrance/17137354148',
 'concentration': 'EDP',
 'size': 50,
 'price': '199,000.00',
 'price_per_ml': '3,980.00',
 'top_notes1': 'Fresh Watery Accord',
 'top_notes2': 'Rose',
 'top_notes3': 'Pink Pepper',
 'mid_notes1': 'Violet',
 'mid_notes2': 'Orange Blossom',
 'mid_notes3': 'Peony',
 'base_notes1': 'Musk',
 'base_notes2': 'Marine',
 'base_notes3': 'Sandalwood',
 'top notes': ['Fresh Watery Accord', 'Rose', 'Pink Pepper']}

accumulate middle fields and using the aggregation pipeline merge it with Extra02

In [66]:
result = extra02_collection.aggregate(accumulate_middle_notes_pipeline)

In [67]:
extra02_collection.find_one()

{'_id': ObjectId('65c3ecc62be6a0a6d64f6747'),
 'brand': 'Alchemist',
 'instagram_link': 'https://www.instagram.com/alchemist.fragrance/',
 'followers': 17800,
 'year': 2022,
 'variant': 'Powder Room',
 'variant_link': 'https://shopee.co.id/alchemist.fragrance/17137354148',
 'concentration': 'EDP',
 'size': 50,
 'price': '199,000.00',
 'price_per_ml': '3,980.00',
 'top_notes1': 'Fresh Watery Accord',
 'top_notes2': 'Rose',
 'top_notes3': 'Pink Pepper',
 'mid_notes1': 'Violet',
 'mid_notes2': 'Orange Blossom',
 'mid_notes3': 'Peony',
 'base_notes1': 'Musk',
 'base_notes2': 'Marine',
 'base_notes3': 'Sandalwood',
 'top notes': ['Fresh Watery Accord', 'Rose', 'Pink Pepper'],
 'middle notes': ['Violet', 'Orange Blossom', 'Peony']}

accumulate base fields and using the aggregation pipeline merge it with Extra02

In [68]:
result = extra02_collection.aggregate(accumulate_base_notes_pipeline)

In [69]:
extra02_collection.find_one()

{'_id': ObjectId('65c3ecc62be6a0a6d64f6747'),
 'brand': 'Alchemist',
 'instagram_link': 'https://www.instagram.com/alchemist.fragrance/',
 'followers': 17800,
 'year': 2022,
 'variant': 'Powder Room',
 'variant_link': 'https://shopee.co.id/alchemist.fragrance/17137354148',
 'concentration': 'EDP',
 'size': 50,
 'price': '199,000.00',
 'price_per_ml': '3,980.00',
 'top_notes1': 'Fresh Watery Accord',
 'top_notes2': 'Rose',
 'top_notes3': 'Pink Pepper',
 'mid_notes1': 'Violet',
 'mid_notes2': 'Orange Blossom',
 'mid_notes3': 'Peony',
 'base_notes1': 'Musk',
 'base_notes2': 'Marine',
 'base_notes3': 'Sandalwood',
 'top notes': ['Fresh Watery Accord', 'Rose', 'Pink Pepper'],
 'middle notes': ['Violet', 'Orange Blossom', 'Peony'],
 'base notes': ['Musk', 'Marine', 'Sandalwood']}

# Project and Integrate the required fields

First we Compare the Extra02 and Fragrances datasets to determine the integration process

In [73]:
pprint(fragrances_collection.find_one())

{'_id': ObjectId('65a54ac72b32912eefb9ea54'),
 'base notes': ['Civet', 'Resins', 'Musk'],
 'company': 'Bogue',
 'description': 'Maai by Bogue is a Floral Woody Musk fragrance for women and '
                'men. Maai was launched in 2014. The nose behind this '
                'fragrance is Antonio Gardoni. Top notes are Aldehydes and '
                'Tuberose; middle notes are Jasmine and Rose; base notes are '
                'Civet, Resins and Musk. .',
 'gender': 'for women and men',
 'gender_vote': {'average': 0.2, 'votes_count': 130},
 'image': 'https://fimgs.net/mdimg/perfume/375x500.26681.jpg',
 'longevity': {'eternal': 69,
               'long lasting': 140,
               'moderate': 28,
               'very weak': 23,
               'weak': 9},
 'main accords': {'aldehydic': 71.419,
                  'amber': 70.8567,
                  'animalic': 80.9679,
                  'balsamic': 44.2436,
                  'fresh': 57.3066,
                  'musky': 100.0,
        

In [82]:
res = extra02_collection.find().limit(5)
for i in range(5):
    pprint(res.next())

{'_id': ObjectId('65c3ecc62be6a0a6d64f6747'),
 'base notes': ['Musk', 'Marine', 'Sandalwood'],
 'base_notes1': 'Musk',
 'base_notes2': 'Marine',
 'base_notes3': 'Sandalwood',
 'brand': 'Alchemist',
 'concentration': 'EDP',
 'followers': 17800,
 'instagram_link': 'https://www.instagram.com/alchemist.fragrance/',
 'mid_notes1': 'Violet',
 'mid_notes2': 'Orange Blossom',
 'mid_notes3': 'Peony',
 'middle notes': ['Violet', 'Orange Blossom', 'Peony'],
 'price': '199,000.00',
 'price_per_ml': '3,980.00',
 'size': 50,
 'top notes': ['Fresh Watery Accord', 'Rose', 'Pink Pepper'],
 'top_notes1': 'Fresh Watery Accord',
 'top_notes2': 'Rose',
 'top_notes3': 'Pink Pepper',
 'variant': 'Powder Room',
 'variant_link': 'https://shopee.co.id/alchemist.fragrance/17137354148',
 'year': 2022}
{'_id': ObjectId('65c3ecc62be6a0a6d64f6748'),
 'base notes': [],
 'brand': 'Alien Objects',
 'concentration': 'EDP',
 'followers': 15700,
 'instagram_link': 'https://www.instagram.com/alienobjects/',
 'mid_notes1': 

In [125]:
integration_pipeline = []

Buid a translation document that translates the fields of Extra02 to Fragrances

In [126]:
translation_doc = {}

In [127]:
translation_doc["company"] = "$brand"

In [128]:
translation_doc["base notes"] = "$base notes"

In [129]:
translation_doc["middle notes"] = "$base notes"

In [130]:
translation_doc["top notes"] = "$top notes"

In [131]:
translation_doc["price"] = "$price"

In [132]:
translation_doc["name"] = "$variant"

In [133]:
translation_doc["year"] = "$year"

In [134]:
translation_doc["link"] = ["$instagram_link", "$variant_link"]

Run a project on the Extra02 using translation to test performance

In [135]:
translate_columns_stage = {
    "$project":translation_doc
}

In [136]:
translate_columns_stage

{'$project': {'company': '$brand',
  'base notes': '$base notes',
  'middle notes': '$base notes',
  'top notes': '$top notes',
  'price': '$price',
  'name': '$variant',
  'year': '$year',
  'link': ['$instagram_link', '$variant_link']}}

In [137]:
res = extra02_collection.aggregate([translate_columns_stage])
for i in range(5):
    pprint(res.next())

{'_id': ObjectId('65c3ecc62be6a0a6d64f6747'),
 'base notes': ['Musk', 'Marine', 'Sandalwood'],
 'company': 'Alchemist',
 'link': ['https://www.instagram.com/alchemist.fragrance/',
          'https://shopee.co.id/alchemist.fragrance/17137354148'],
 'middle notes': ['Musk', 'Marine', 'Sandalwood'],
 'name': 'Powder Room',
 'price': '199,000.00',
 'top notes': ['Fresh Watery Accord', 'Rose', 'Pink Pepper'],
 'year': 2022}
{'_id': ObjectId('65c3ecc62be6a0a6d64f6748'),
 'base notes': [],
 'company': 'Alien Objects',
 'link': ['https://www.instagram.com/alienobjects/',
          'https://shopee.co.id/alienobjects/14283008217'],
 'middle notes': [],
 'name': 'XOXO',
 'price': '750,000.00',
 'top notes': ['Dew', 'Strawberry', 'Lotus', 'Tulip', 'Musk', 'Oakmoss'],
 'year': 2019}
{'_id': ObjectId('65c3ecc62be6a0a6d64f6749'),
 'base notes': ['Vanilla', 'Caramel', 'Musk', 'Cedarwood'],
 'company': 'ALT Perfumery',
 'link': ['https://www.instagram.com/alt.perfumery/',
          'https://shopee.co.i

Do the final merge/integration into fragrances

In [138]:
merge_stage = {"$merge": {"into": "Fragrances"}}

In [139]:
integration_pipeline.extend([translate_columns_stage, merge_stage])

In [140]:
integration_pipeline

[{'$project': {'company': '$brand',
   'base notes': '$base notes',
   'middle notes': '$base notes',
   'top notes': '$top notes',
   'price': '$price',
   'name': '$variant',
   'year': '$year',
   'link': ['$instagram_link', '$variant_link']}},
 {'$merge': {'into': 'Fragrances'}}]

In [141]:
fragrances_collection.count_documents({})

4563

In [142]:
extra02_collection.aggregate(integration_pipeline)

In [143]:
fragrances_collection.count_documents({})

4563

In [144]:
client.close()

Done!